In [ ]:
%matplotlib inline
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import src.dab_util as du

In [ ]:
import src.signal_gen as sg
reload(sg)
reload(du)

In [ ]:
sg.gen_ramps(amplitudes=np.linspace(0.001, 0.95, num = 20))

In [ ]:
a_in  = np.fromfile("./input.dat", dtype=np.complex64)
a_out = np.fromfile("./output.dat", dtype=np.complex64)

In [ ]:
def crop_signal(signal, n_window = 1000, n_zeros = 50000, debug = False):
    #signal = signal[-10:-1]
    mag = abs(signal)
    window = np.ones(n_window) / float(n_window)
    mag = scipy.signal.convolve(window, mag)
    mag = scipy.signal.convolve(window, mag)
    thr = 0.01 * np.max(mag)
    idx_start = np.argmax(mag > thr)
    idx_end = mag.shape[0] - np.argmax(np.flipud(mag > thr))
    if debug:
        plt.plot(mag < thr)
        plt.plot((idx_start,idx_start), (0,0.1), color='g', linewidth=2)
        plt.plot((idx_end,idx_end), (0,0.1), color='r', linewidth=2)
    signal = signal[idx_start - n_zeros: idx_end + n_zeros]
    return signal

In [ ]:
a_in  = du.crop_signal(a_in)

In [ ]:
a_out = du.crop_signal(a_out)

In [ ]:
#plt.plot(a_in.real[780000 + 230000:2000000]+1, color='b');
#plt.plot(a_out.real[780000:2000000], color='g');
plt.plot(a_in.real+1, color='b');
plt.plot(a_out.real, color='g');

In [ ]:
def lagcorr(x,y,lag=None,verbose=True):
    '''Compute lead-lag correlations between 2 time series.

    <x>,<y>: 1-D time series.
    <lag>: lag option, could take different forms of <lag>:
          if 0 or None, compute ordinary correlation and p-value;
          if positive integer, compute lagged correlation with lag
          upto <lag>;
          if negative integer, compute lead correlation with lead
          upto <-lag>;
          if pass in an list or tuple or array of integers, compute 
          lead/lag correlations at different leads/lags.

    Note: when talking about lead/lag, uses <y> as a reference.
    Therefore positive lag means <x> lags <y> by <lag>, computation is
    done by shifting <x> to the left hand side by <lag> with respect to
    <y>.
    Similarly negative lag means <x> leads <y> by <lag>, computation is
    done by shifting <x> to the right hand side by <lag> with respect to
    <y>.

    Return <result>: a (n*2) array, with 1st column the correlation 
    coefficients, 2nd column correpsonding p values.

    Currently only works for 1-D arrays.
    '''

    import numpy
    from scipy.stats import pearsonr

    if len(x)!=len(y):
        raise Exception('Input variables of different lengths.')

    #--------Unify types of <lag>-------------
    if numpy.isscalar(lag):
        if abs(lag)>=len(x):
            raise Exception('Maximum lag equal or larger than array.')
        if lag<0:
            lag=-numpy.arange(abs(lag)+1)
        elif lag==0:
            lag=[0,]
        else:
            lag=numpy.arange(lag+1)    
    elif lag is None:
        lag=[0,]
    else:
        lag=numpy.asarray(lag)

    #-------Loop over lags---------------------
    result=[]
    if verbose:
        print '\n#<lagcorr>: Computing lagged-correlations at lags:',lag

    for ii in lag:
        if ii<0:
            result.append(pearsonr(x[:ii],y[-ii:]))
        elif ii==0:
            result.append(pearsonr(x,y))
        elif ii>0:
            result.append(pearsonr(x[ii:],y[:-ii]))

    result=numpy.asarray(result)

    return result

In [ ]:
l = min(a_out.shape[0], a_in.shape[0])
a_out = a_out[0:l]
a_in  = a_in[0:l]

c = lagcorr(abs(a_out), abs(a_in), 1000)[:,0]

In [ ]:
#def corr(a_out, a_in):
#    import scipy
#    corr = np.correlate(
#        a_out,
#        a_in,
#        'full')
#    delay = np.argmax(corr) - a_in.shape[0] + 1
#    #plt.plot(range(-corr.shape[0]/2, corr.shape[0]/2),corr)
#    return delay

In [ ]:
#delay = corr(a_out, a_in)
delay = np.argmax(c)
a_out = a_out[delay - 1:]
delay

In [ ]:
l = min(a_out.shape[0], a_in.shape[0])
a_out = a_out[0:l]
a_in  = a_in[0:l]

In [ ]:
plt.plot(a_in, color='g');
plt.plot(a_out - 1, color='y');

In [ ]:
def get_amp_ratio(ampl_1, ampl_2):
    idxs = (a_in > ampl_1) & (a_in < ampl_2)
    ratio = (a_out[idxs] / a_in[idxs])
    return ratio.mean(), ratio.var()

In [ ]:
def get_phase(ampl_1, ampl_2):
    idxs = (a_in > ampl_1) & (a_in < ampl_2)
    ratio = np.angle(a_out[idxs]) - np.angle(a_in[idxs])
    return ratio.mean(), ratio.var()

In [ ]:
bins = np.linspace(0.1,1,num=10)
res = []
for ampl_1, ampl_2 in zip(bins, bins[1:]):
    res.append(get_amp_ratio(ampl_1, ampl_2))
mean, var = zip(*res)

In [ ]:
plt.plot(mean)

In [ ]:
bins = np.linspace(0.1,1,num=10)
res = []
for ampl_1, ampl_2 in zip(bins, bins[1:]):
    res.append(get_phase(ampl_1, ampl_2))
mean, var = zip(*res)

In [ ]:
plt.plot(mean)